In [4]:
#importing necessary modules
from datetime import datetime as dt
import pandas as pd
import numpy as np
import xarray as xr

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.pyplot as plt

import metpy.calc as mpcalc
import metpy.plots as mpplots

from matplotlib.patheffects import withStroke
from metpy.io import parse_metar_file
from metpy.units import pandas_dataframe_to_unit_arrays
from metpy.cbook import get_test_data
from metpy.units import units
 
from siphon.catalog import TDSCatalog

# Ignore warnings
import warnings

warnings.filterwarnings('ignore')

In [5]:
#Link to datafile: https://drive.google.com/file/d/1kjzuaNx7G6EmjFbIToN6DitoZnbmfy9V/view?usp=sharing

In [6]:
# Open the netCDF file as a xarray Dataset
data = xr.open_dataset('C:\\Users\\Dell\\Documents\\BD final proj\\projdata1.nc4', decode_times = False)

# View a summary of the Dataset
data

ValueError: File 'projdata1.nc4' is not in the registry.